In [ ]:
# import os
# os.environ["JAX_PLATFORM_NAME"] = "cpu"

import jax_sbgeom as jsb
%load_ext autoreload
%autoreload 2
import jax.numpy as jnp
import jax
jax.config.update("jax_enable_x64", True)
import numpy as onp
import sys 
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)

import StellBlanket.SBGeom as SBGeom
from functools import partial
import jax_sbgeom.flux_surfaces.flux_surface_meshing as fsm
import jax_sbgeom.flux_surfaces.flux_surfaces_base as fsb
import jax_sbgeom.flux_surfaces.flux_surfaces_extended as fse
from jax_sbgeom.jax_utils.utils import _mesh_to_pyvista_mesh, _vertices_to_pyvista_polyline
import pyvista as pv
from dataclasses import dataclass
import h5py
import jax_sbgeom
import matplotlib.pyplot as plt


def _get_coil_and_fs_files():
#    return [()"/home/tbogaarts/stellarator_paper/base_data/vmecs/HELIAS3_coils_all.h5", "/home/tbogaarts/stellarator_paper/base_data/vmecs/HELIAS5_coils_all.h5", "/home/tbogaarts/stellarator_paper/base_data/vmecs/squid_coilset.h5"]
    return [("/home/tbogaarts/stellarator_paper/base_data/vmecs/helias3_vmec.nc4", "/home/tbogaarts/stellarator_paper/base_data/vmecs/HELIAS3_coils_all.h5"), ("/home/tbogaarts/stellarator_paper/base_data/vmecs/helias5_vmec.nc4", "/home/tbogaarts/stellarator_paper/base_data/vmecs/HELIAS5_coils_all.h5"), ("/home/tbogaarts/stellarator_paper/base_data/vmecs/squid_vmec.nc4", "/home/tbogaarts/stellarator_paper/base_data/vmecs/squid_coilset.h5")]

def _get_discrete_coils(coil_file):
    import h5py
    with h5py.File(coil_file, 'r') as f:
        coil_data = jnp.array(f['Dataset1'])
    return jsb.coils.CoilSet.from_list([jsb.coils.DiscreteCoil.from_positions(coil_data[i]) for i in range(coil_data.shape[0])])

#@pytest.fixture(scope="session", params = _get_coil_and_fs_files())
def _get_cws_and_flux_surface(request):
    fs_jax = jsb.flux_surfaces.FluxSurfaceNormalExtendedNoPhi.from_hdf5(request.param[0])
    coilset_jax = _get_discrete_coils(request.param[1])
    cws_mesh = jsb.coils.coil_winding_surface.create_optimized_coil_winding_surface(coilset_jax, 100, 200, 'spline')
    return cws_mesh, fs_jax
class req:
    def __init__(self, param):
        self.param = param
    
cws_mesh, fs_jax = _get_cws_and_flux_surface(req(_get_coil_and_fs_files()[0]))


In [ ]:
fs_j2 = jsb.flux_surfaces.FluxSurfaceNormalExtendedConstantPhi.from_hdf5("/home/tbogaarts/stellarator_paper/base_data/vmecs/helias3_vmec.nc4")

fs_j2.cartesian_position(1.0, jnp.array([0.1]), jnp.array([0.2]))

In [ ]:
import matplotlib.pyplot as plt
theta, phi, dmesh = jsb.flux_surfaces.flux_surfaces_utilities.generate_thickness_matrix(fs_jax, cws_mesh, 210, 100)
plt.pcolormesh(phi, theta, dmesh)
plt.colorbar()

In [ ]:
phi = 0.0
theta = jnp.linspace(0, 2 * jnp.pi, 100)
positions_cartesian = fs_jax.cartesian_position(1.0,theta,phi)

In [ ]:

from jax_sbgeom.flux_surfaces.flux_surfaces_base import _arc_length_theta


In [ ]:
arc_length_fourier = _arc_length_theta(fs_jax.data, fs_jax.settings, 2.0, theta, phi)

In [ ]:
arc_length = jnp.sqrt(jnp.sum(jnp.diff(positions_cartesian, axis=0)**2, axis=-1)) / jnp.diff(theta)
cum_length = jnp.concatenate([jnp.array([0.0]), jnp.cumsum(arc_length)])
#plt.plot(cum_length,'ro-')    
plt.plot(arc_length,'ro-')    
plt.plot(arc_length_fourier,'b-')

In [ ]:
rz = jnp.sqrt(positions_cartesian[:,0]**2 + positions_cartesian[:,1]**2)
plt.plot(rz, positions_cartesian[:,2],'ro-')

In [ ]:
positions = fs_jax.cartesian_position(1.0 + dmesh, theta, phi )

In [ ]:
plotter = pv.Plotter()

plotter.add_mesh(_mesh_to_pyvista_mesh(*cws_mesh), color='lightblue',  show_edges=True)
plotter.add_mesh(pv.PolyData(onp.array(positions).reshape(-1,3)), color='red', point_size=20, render_points_as_spheres=True)
plotter.show()

In [ ]:
from StellBlanket.SBGeom import Coils_jax as CJ

In [ ]:

def _get_all_fourier_coils(request):
    coilset_sbgeom = SBGeom.Coils.Discrete_Coil_Set_From_HDF5(coil_files[request])
    coilset_fourier = SBGeom.Coils.Convert_to_Fourier_Coils(coilset_sbgeom)
    
    coilset_jax = [jsb.coils.FourierCoil(jnp.array(i.Get_Fourier_Cos()), jnp.array(i.Get_Fourier_Sin()), jnp.array(i.Get_Centre())) for i in coilset_fourier]
    return coilset_jax, coilset_fourier

_, sbgeom_coilset = _get_all_fourier_coils(stell_i)

In [ ]:
splines = CJ.Create_Optimized_Splines(sbgeom_coilset, 100, sbgeom_coilset.Number_of_Coils() )
mesh_cws_sbgeom = SBGeom.Mesh.Mesh(splines.reshape(-1,3), SBGeom.Mesh.Connectivity_Toroidal_Surface_Mesh(sbgeom_coilset.Number_of_Coils(), 100, (0.0, 2 * onp.pi)))

In [ ]:
phi, theta, d_matrix = SBGeom.util.Generate_Thickness_Matrix_Raytracing(fs_sbgeom, mesh_cws_sbgeom, 100, 210)

In [ ]:
plt.pcolormesh(phi, theta, d_matrix - dmesh, vmin =-0.2, vmax=0.2, cmap = 'bwr')
plt.colorbar()

In [ ]:
plotter = pv.Plotter()
plotter.add_mesh(mesh_cws_sbgeom.to_pyvista())
plotter.add_mesh(_mesh_to_pyvista_mesh(*cws_mesh), color='red', opacity=1.0)
plotter.show()